Learning curve of the VAEs for the IMGEP-OGL approach. (Fig. 16, Supplementary Material)

In [ ]:
# default print properties
multiplier = 2

pixel_cm_ration = 36.5

width_full = int(13.95 * pixel_cm_ration) * multiplier
width_half = int(13.95/2 * pixel_cm_ration) * multiplier
width_third = int(13.95/3 * pixel_cm_ration) * multiplier

height_default_1 = int(3.5 * pixel_cm_ration) * multiplier

# margins in pixel
top_margin = 0 * multiplier 
left_margin = 35 * multiplier 
right_margin = 0 * multiplier 
bottom_margin = 25 * multiplier 

font_size = 10 * multiplier 
font_family='Times New Roman'

line_width = 2 * multiplier 

In [ ]:
# Config
import autodisc as ad
import ipywidgets
import plotly
import numpy as np
plotly.offline.init_notebook_mode(connected=True)

default_config = dict(
    plotly_format = 'svg',
    layout = dict(
        xaxis = dict(
            title = 'epoch',
            showline = True,
            linewidth = 1,
            zeroline=False,
            ),
        yaxis = dict(
            title = 'loss',
            showline = True,
            linewidth = 1,
            zeroline=False,
            ),
        font = dict(
            family=font_family, 
            size=font_size, 
            ),
        updatemenus=[],
        width=width_third, # in cm
        height=height_default_1, # in cm
        
        margin = dict(
            l=left_margin, #left margin in pixel
            r=right_margin, #right margin in pixel
            b=bottom_margin, #bottom margin in pixel
            t=top_margin,  #top margin in pixel
            ),

        legend=dict(
            xanchor='right',
            yanchor='top',
            y=1,
            x=1,
            ),        
        ),
    
    default_colors = ['rgb(0,0,0)', 
                      #'rgb(204,121,167)', 
                      'rgb(230,159,0)', 
                      'rgb(0,158,115)',
                      'rgb(240,228,66)',
                      'rgb(213,94,0)', 
                      'rgb(0,0,0)',  'rgb(86,180,233)', 'rgb(0,158,115)', 'rgb(240,228,66)', 'rgb(0,114,178)', 'rgb(213,94,0)', 'rgb(204,121,167)'],
    
    default_mean_trace = dict(line=dict(width = line_width)),
    
    mean_traces = [
        dict(line = dict(dash = 'solid')),
        dict(line = dict(dash = 'solid')),
        dict(line = dict(dash = 'dash')),
        dict(line = dict(dash = 'dashdot')),
        dict(line = dict(dash = 'solid')),
        dict(line = dict(dash = 'longdashdot')),
        dict(line = dict(dash = 'longdash')),
        dict(line = dict(dash = 'solid')),
        dict(line = dict(dash = 'dash')),
        dict(line = dict(dash = 'dashdot')),
        dict(line = dict(dash = 'dot')),
        dict(line = dict(dash = 'longdash')),
        dict(line = dict(dash = 'longdashdot')),
    ],
   
)

In [ ]:
# get data
import os

def load_repetition_training_data_from_csv_files(repetition_directory_path):
    
    # get training loss for each repetition
    train_losses = {}
    with open(os.path.join(repetition_directory_path, 'trained_representation', 'training_analysis', 'loss_train.csv'), 'r') as f:
        lineslist = [line.rstrip() for line in f]
        for line in lineslist:
            line = line.split('\t')
            if line[0][:6] == 'Epoch:':
                for col in range(1, len(line)):
                    k,v = line[col].split(' ')
                    k = k[:-1]
                    if k not in train_losses:
                            train_losses[k] = [float(v)]
                    else:
                        train_losses[k].append(float(v))
                
    #
    valid_losses = {}
    with open(os.path.join(repetition_directory_path, 'trained_representation', 'training_analysis', 'loss_valid.csv'), 'r') as f:
        lineslist = [line.rstrip() for line in f]
        for line in lineslist:
            line = line.split('\t')
            # note: the code to produce the csv has an error and did not add 'Epoch:' before the epoch number
            for col in range(1, len(line)):
                k,v = line[col].split(' ')
                if k and v:
                    k = k[:-1]
                    if k not in valid_losses:
                            valid_losses[k] = [float(v)]
                    else:
                        valid_losses[k].append(float(v))

    return [train_losses, valid_losses] 


def load_vae_training_data_from_csv_files(experiment_directory_path):
    
    # identify the repetition folders
    repetition_directories = []
    repetition_id = 0
    while True:
        
        repetition_directory_path = os.path.join(experiment_directory_path,'repetition_{:06d}'.format(repetition_id))
        
        if os.path.isdir(repetition_directory_path):
            repetition_directories.append(repetition_directory_path)
            repetition_id += 1
        else:
            break

    # load individuall losses per repetition
    train_losses = []
    valid_losses = []
    
    for repetition_directory_path in repetition_directories:
        [rep_train_losses, rep_valid_losses] = load_repetition_training_data_from_csv_files(repetition_directory_path)
        
        train_losses.append(rep_train_losses['total'])
        valid_losses.append(rep_valid_losses['total'])
        
    train_losses = np.array(train_losses)
    valid_losses = np.array(valid_losses)
    
    return [train_losses, valid_losses]

# IMGEP OGL - VAE

In [ ]:
# get data
[vae_train_losses, vae_valid_losses] = load_vae_training_data_from_csv_files('../experiments/experiment_000301')

In [ ]:
import os
import autodisc

config = default_config
config['mean_labels'] = ['training set', 'validation set']
config['layout']['showlegend'] = False
config['layout']['yaxis']['range'] = [0, 90000]
config['layout']['xaxis']['range'] = [0, 1999]

data = []
data.append(vae_train_losses)
data.append(vae_valid_losses)

fig = autodisc.gui.jupyter.plotly_meanstd_scatter(data=data, config=config)

# IMGEP OGL - beta VAE

In [ ]:
# get data
[bvae_train_losses, bvae_valid_losses] = load_vae_training_data_from_csv_files('../experiments/experiment_000302')

In [ ]:
import os
import autodisc

config = default_config
config['mean_labels'] = ['training set', 'validation set']
config['layout']['showlegend'] = False
config['layout']['yaxis']['range'] = [0, 90000]
config['layout']['xaxis']['range'] = [0, 1999]

data = []
data.append(bvae_train_losses)
data.append(bvae_valid_losses)

fig = autodisc.gui.jupyter.plotly_meanstd_scatter(data=data, config=config)

# IMGEP OGL - beta VAE augmented

In [ ]:
# get data
[bvaeaug_train_losses, bvaeaug_valid_losses] = load_vae_training_data_from_csv_files('../experiments/experiment_000303')

In [ ]:
import os
import autodisc

config = default_config
config['mean_labels'] = ['training set', 'validation set']
config['layout']['showlegend'] = True
config['layout']['yaxis']['range'] = [0, 90000]
config['layout']['xaxis']['range'] = [0, 1999]
data = []
data.append(bvaeaug_train_losses)
data.append(bvaeaug_valid_losses)

fig = autodisc.gui.jupyter.plotly_meanstd_scatter(data=data, config=config)